# 一次元CNNによる学習と識別性能評価（時系列解析）

---
  
引数：raw_rest.csv/raw_tapping.csvがあるディレクトリまでのパス  
  
---
  
入力：raw_rest.csv/raw_tapping.csv

---
  
出力：ACCURACY[loo or k_cv]_timeseries_1dCNN.csv　識別性能評価結果一覧  

---  
  
1dCNNを用いて学習し，交差検証法（k-分割交差検証，leave-one-out交差検証）を用いて識別性能評価を行う．  
ベクトル：各ボクセルの時系列データ

In [16]:
print('########### ML_1dCNN_timeseries.py program excution ############')

########### ML_1dCNN_timeseries.py program excution ############


In [2]:
import pandas as pd
import sys

# matplotlib inline部分は.pyの時にはコメントアウトしないとエラー出る！

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
%matplotlib inline
import matplotlib.pyplot as plt
from keras.layers.convolutional import Conv1D, UpSampling1D
from keras.layers.pooling import MaxPooling1D
from keras.utils import np_utils

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


コマンドライン引数でraw_rest.csv/raw_tapping.csvファイルがあるディレクトリまでのパスを取得

In [8]:
#args = sys.argv
#PATH = args[1]

# jupyter notebookのときはここで指定
PATH = '../State-2fe_Active/20181029rn/64ch/RawData/'

# 検証手法
col_name = 'leave-one-out'

## ONEdCNN_LOO関数

引数として教師データをX，ラベルをyで受け取る．  
交差検証法の一つleave-one-out交差検証で識別精度評価を行う．

* (1個をテストデータ，残りを教師データにして学習・評価) * すべてのデータ個
* 得られたすべてのデータ個の評価結果（識別率）の平均を求めてパーセントに直す
* 評価結果（識別率）をmain関数に返す

In [9]:
def ONEdCNN_LOO(X, y):

    # 識別率を格納する配列
    LOOscore = np.zeros(len(X))
    
    # ベクトルの長さを格納しておく
    bach_size = X.shape[1]

    
    # 1個をテストデータ，残りを教師データにして学習・評価
    # すべてのデータに対して行う
    for i in range(len(X)):
        
        print('------ ' + str(i) + ' / ' + str(len(X)) + '回 -----')

        # テストデータ
        X_test = X[i]
        y_test = y[i]

        # テストデータとして使用するデータを除いた教師データを作成
        X_train = np.delete(X, i, 0)
        y_train = np.delete(y, i, 0)


        # （データ数, ベクトルの長さ，1）という形にリシェイプする
        X_train = np.reshape(X_train, (-1, bach_size, 1))
        X_test = np.reshape(X_test, (-1, bach_size, 1))
        
        # ダミー変数に変換：何分類するかによって数字を書き換える（rest, Tappingの2分類なので2）
        y_train = np_utils.to_categorical(y_train, 2)
        y_test = np_utils.to_categorical(y_test, 2)


        # 1次元CNNのインスタンスを作成
        # 参考文献（Time series classification via TDA）に従ってパラメータを設定
        # 1st Conv1d : kernel_number = 7, kernel_size = 6
        # 1st MacPooling : kernel_number = 7
        # 2nd Conv1d : kernel_number = 7, kernel_size = 2
        # 2nd MacPooling : kernel_number = 3
        # Flattenで1次元に
        model = Sequential()
        model.add(Conv1D(7, 6, padding='same', input_shape=(bach_size, 1)))
        model.add(Activation('relu'))
        model.add(MaxPooling1D(7, padding='same'))
        model.add(Conv1D(7, 2, padding='same', activation='relu'))
        model.add(MaxPooling1D(3, padding='same'))

        model.add(Flatten())
        
        # 何分類するかによってunitsを書き換える（rest, Tappingの2分類なのでunits=2）
        model.add(Dense(units=2, activation='softmax'))
        model.compile(loss='mse', optimizer='adam', metrics=["accuracy"])

        # 学習
        history = model.fit(X_train, y_train, epochs=100)
        
        # 評価結果（識別率）を格納
        score = model.evaluate(X_test, y_test)
        LOOscore[i] = score[1]
        

        
    # 評価結果（識別率）の平均を求める
    result = LOOscore.mean()
    
    # パーセントに直す
    result = round(result * 100, 1)
    
    print(str(LOOscore) + '\n')

    return result

## ONEdCNN_kCV関数

引数として教師データをX，ラベルをyで受け取る．  
交差検証法の一つk-分割交差検証で識別精度評価を行う．

* 学習
* (k分割し，1グループをテストデータ，残りグループを教師データにして評価) * k
* 得られたk個の評価結果（識別率）の平均を求めてパーセントに直す
* 評価結果（識別率）をmain関数に返す

In [10]:
def ONEdCNN_kCV(X, y):
    
    
    # 識別率を格納する配列
    kCVscore = np.zeros(cv_k)
    
    # ベクトルの長さを格納しておく
    bach_size = X.shape[1]
    
    # 分割数
    kf = KFold(n_splits=cv_k, shuffle=False)
    # 繰り返し回数
    i = 0

    for train_index, eval_index in kf.split(X):
        
        print('------ ' + str(i) + ' / ' + str(cv_k) + '回 -----')
        
        X_train, X_test = X[train_index], X[eval_index]
        y_train, y_test = y[train_index], y[eval_index]

        # （データ数, ベクトルの長さ，1）という形にリシェイプする
        X_train = np.reshape(X_train, (-1, bach_size, 1))
        X_test = np.reshape(X_test, (-1, bach_size, 1))

        # ダミー変数に変換：何分類するかによって数字を書き換える（rest, Tappingの2分類なので2）
        y_train = np_utils.to_categorical(y_train, 2)
        y_test = np_utils.to_categorical(y_test, 2)


        # 1次元CNNのインスタンスを作成
        # 参考文献（Time series classification via TDA）に従ってパラメータを設定
        # 1st Conv1d : kernel_number = 7, kernel_size = 6
        # 1st MacPooling : kernel_number = 7
        # 2nd Conv1d : kernel_number = 7, kernel_size = 2
        # 2nd MacPooling : kernel_number = 3
        # Flattenで1次元に
        model = Sequential()
        model.add(Conv1D(7, 6, padding='same', input_shape=(bach_size, 1)))
        model.add(Activation('relu'))
        model.add(MaxPooling1D(7, padding='same'))
        model.add(Conv1D(7, 2, padding='same', activation='relu'))
        model.add(MaxPooling1D(3, padding='same'))

        model.add(Flatten())
        
        # 何分類するかによってunitsを書き換える（rest, Tappingの2分類なのでunits=2）
        model.add(Dense(units=2, activation='softmax'))
        model.compile(loss='mse', optimizer='adam', metrics=["accuracy"])

        # 学習
        history = model.fit(X_train, y_train, epochs=100)
        
        # 評価結果（識別率）を格納
        score = model.evaluate(X_test, y_test)
        kCVscore[i] = score[1]
        
        i = i + 1
        

    # 評価結果（識別率）の平均を求める
    result = kCVscore.mean()
    
    # パーセントに直す
    result = round(result * 100, 1)
    
    print('k = ' + str(cv_k) + '：' + str(kCVscore))

    return result

## TrainingData関数 
引数として読み込みたいタスクごとのファイル名をfile_rest/file_tappingで受け取る． 
* 機械学習にかけれるようにデータのベクトル化とラベルを作成
* ベクトル化したデータとラベルをmain関数に返す

In [11]:
def TrainingData(file_rest, file_tapping):
    
    # 読み込みたいファイルのパス
    PATH_rest = PATH + file_rest
    PATH_tapping = PATH + file_tapping

    # csvファイル読み込み（転地することでベクトルを時系列データとする）
    rest = pd.read_csv(PATH_rest, header = 0)
    rest = rest.T
    
    tapping = pd.read_csv(PATH_tapping, header = 0)
    tapping = tapping.T

    # 各タスクのデータを縦結合
    all_data = pd.concat([rest, tapping], axis = 0)

    # ベクトル化
    X = all_data.as_matrix()

    # ラベル作成 rock = 0, scissor = 1, paper = 2
    label_rest = np.zeros(len(rest.index))
    label_tapping = np.ones(len(tapping.index))

    y = np.r_[label_rest, label_tapping]

    
    return X, y

## main関数

In [12]:
if __name__ == '__main__':
    
    # データとラベルの準備
    data, labels = TrainingData('raw_rest.csv', 'raw_tapping.csv')


In [13]:
    # 学習とleave-one-out交差検証法
    
    print('leave-one-out Cross-Validation')
    
    result_loo = ONEdCNN_LOO(data, labels)
    
    # データフレーム化
    result_loo = pd.DataFrame({col_name:[result_loo]}, index = ['TimeSeries + 1dCNN'])
    print(result_loo)

leave-one-out Cross-Validation
------ 0 / 14回 -----
Epoch 1/100
13/13 [==============================] - 0s 8ms/step - loss: 0.2505 - acc: 0.4615
Epoch 2/100
13/13 [==============================] - 0s 318us/step - loss: 0.2488 - acc: 0.4615
Epoch 3/100
13/13 [==============================] - 0s 274us/step - loss: 0.2473 - acc: 0.4615
Epoch 4/100
13/13 [==============================] - 0s 261us/step - loss: 0.2457 - acc: 0.4615
Epoch 5/100
13/13 [==============================] - 0s 282us/step - loss: 0.2440 - acc: 0.4615
Epoch 6/100
13/13 [==============================] - 0s 240us/step - loss: 0.2423 - acc: 0.4615
Epoch 7/100
13/13 [==============================] - 0s 303us/step - loss: 0.2406 - acc: 0.4615
Epoch 8/100
13/13 [==============================] - 0s 205us/step - loss: 0.2388 - acc: 0.4615
Epoch 9/100
13/13 [==============================] - 0s 296us/step - loss: 0.2370 - acc: 0.4615
Epoch 10/100
13/13 [==============================] - 0s 355us/step - loss: 0.2352 - a

13/13 [==============================] - 0s 308us/step - loss: 0.1304 - acc: 1.0000
Epoch 86/100
13/13 [==============================] - 0s 274us/step - loss: 0.1290 - acc: 1.0000
Epoch 87/100
13/13 [==============================] - 0s 239us/step - loss: 0.1276 - acc: 1.0000
Epoch 88/100
13/13 [==============================] - 0s 292us/step - loss: 0.1262 - acc: 1.0000
Epoch 89/100
13/13 [==============================] - 0s 235us/step - loss: 0.1248 - acc: 1.0000
Epoch 90/100
13/13 [==============================] - 0s 258us/step - loss: 0.1234 - acc: 1.0000
Epoch 91/100
13/13 [==============================] - 0s 296us/step - loss: 0.1221 - acc: 1.0000
Epoch 92/100
13/13 [==============================] - 0s 323us/step - loss: 0.1208 - acc: 1.0000
Epoch 93/100
13/13 [==============================] - 0s 235us/step - loss: 0.1194 - acc: 1.0000
Epoch 94/100
13/13 [==============================] - 0s 311us/step - loss: 0.1182 - acc: 1.0000
Epoch 95/100
13/13 [=======================

Epoch 69/100
13/13 [==============================] - 0s 255us/step - loss: 0.1771 - acc: 0.8462
Epoch 70/100
13/13 [==============================] - 0s 372us/step - loss: 0.1760 - acc: 0.8462
Epoch 71/100
13/13 [==============================] - 0s 358us/step - loss: 0.1748 - acc: 0.8462
Epoch 72/100
13/13 [==============================] - 0s 289us/step - loss: 0.1737 - acc: 0.8462
Epoch 73/100
13/13 [==============================] - 0s 310us/step - loss: 0.1726 - acc: 0.8462
Epoch 74/100
13/13 [==============================] - 0s 319us/step - loss: 0.1715 - acc: 0.8462
Epoch 75/100
13/13 [==============================] - 0s 290us/step - loss: 0.1704 - acc: 0.8462
Epoch 76/100
13/13 [==============================] - 0s 327us/step - loss: 0.1693 - acc: 0.8462
Epoch 77/100
13/13 [==============================] - 0s 417us/step - loss: 0.1682 - acc: 0.8462
Epoch 78/100
13/13 [==============================] - 0s 395us/step - loss: 0.1671 - acc: 0.8462
Epoch 79/100
13/13 [==========

Epoch 53/100
13/13 [==============================] - 0s 257us/step - loss: 0.2280 - acc: 0.6154
Epoch 54/100
13/13 [==============================] - 0s 309us/step - loss: 0.2267 - acc: 0.6154
Epoch 55/100
13/13 [==============================] - 0s 342us/step - loss: 0.2256 - acc: 0.6154
Epoch 56/100
13/13 [==============================] - 0s 259us/step - loss: 0.2244 - acc: 0.6154
Epoch 57/100
13/13 [==============================] - 0s 277us/step - loss: 0.2233 - acc: 0.6154
Epoch 58/100
13/13 [==============================] - 0s 318us/step - loss: 0.2223 - acc: 0.6154
Epoch 59/100
13/13 [==============================] - 0s 297us/step - loss: 0.2212 - acc: 0.6154
Epoch 60/100
13/13 [==============================] - 0s 241us/step - loss: 0.2201 - acc: 0.6154
Epoch 61/100
13/13 [==============================] - 0s 301us/step - loss: 0.2190 - acc: 0.6154
Epoch 62/100
13/13 [==============================] - 0s 225us/step - loss: 0.2180 - acc: 0.6154
Epoch 63/100
13/13 [==========

Epoch 37/100
13/13 [==============================] - 0s 321us/step - loss: 0.2314 - acc: 0.6154
Epoch 38/100
13/13 [==============================] - 0s 240us/step - loss: 0.2307 - acc: 0.6154
Epoch 39/100
13/13 [==============================] - 0s 319us/step - loss: 0.2300 - acc: 0.6154
Epoch 40/100
13/13 [==============================] - 0s 340us/step - loss: 0.2294 - acc: 0.6154
Epoch 41/100
13/13 [==============================] - 0s 382us/step - loss: 0.2287 - acc: 0.6154
Epoch 42/100
13/13 [==============================] - 0s 224us/step - loss: 0.2280 - acc: 0.6154
Epoch 43/100
13/13 [==============================] - 0s 346us/step - loss: 0.2273 - acc: 0.6154
Epoch 44/100
13/13 [==============================] - 0s 304us/step - loss: 0.2267 - acc: 0.6154
Epoch 45/100
13/13 [==============================] - 0s 342us/step - loss: 0.2260 - acc: 0.6154
Epoch 46/100
13/13 [==============================] - 0s 392us/step - loss: 0.2254 - acc: 0.6154
Epoch 47/100
13/13 [==========

...

Epoch 10/100
13/13 [==============================] - 0s 279us/step - loss: 0.2461 - acc: 0.5385
Epoch 11/100
13/13 [==============================] - 0s 354us/step - loss: 0.2431 - acc: 0.5385
Epoch 12/100
13/13 [==============================] - 0s 241us/step - loss: 0.2404 - acc: 0.5385
Epoch 13/100
13/13 [==============================] - 0s 335us/step - loss: 0.2381 - acc: 0.5385
Epoch 14/100
13/13 [==============================] - 0s 245us/step - loss: 0.2361 - acc: 0.5385
Epoch 15/100
13/13 [==============================] - 0s 321us/step - loss: 0.2342 - acc: 0.6154
Epoch 16/100
13/13 [==============================] - 0s 351us/step - loss: 0.2325 - acc: 0.6154
Epoch 17/100
13/13 [==============================] - 0s 263us/step - loss: 0.2309 - acc: 0.6923
Epoch 18/100
13/13 [==============================] - 0s 317us/step - loss: 0.2294 - acc: 0.6923
Epoch 19/100
13/13 [==============================] - 0s 283us/step - loss: 0.2280 - acc: 0.6923
Epoch 20/100
13/13 [==========

Epoch 95/100
13/13 [==============================] - 0s 291us/step - loss: 0.1553 - acc: 0.8462
Epoch 96/100
13/13 [==============================] - 0s 319us/step - loss: 0.1543 - acc: 0.8462
Epoch 97/100
13/13 [==============================] - 0s 338us/step - loss: 0.1534 - acc: 0.8462
Epoch 98/100
13/13 [==============================] - 0s 305us/step - loss: 0.1525 - acc: 0.8462
Epoch 99/100
13/13 [==============================] - 0s 397us/step - loss: 0.1515 - acc: 0.8462
Epoch 100/100
1/1 [==============================] - 0s 180ms/step
------ 11 / 14回 -----
Epoch 1/100
13/13 [==============================] - 0s 38ms/step - loss: 0.2653 - acc: 0.4615
Epoch 2/100
13/13 [==============================] - 0s 300us/step - loss: 0.2621 - acc: 0.5385
Epoch 3/100
13/13 [==============================] - 0s 274us/step - loss: 0.2593 - acc: 0.5385
Epoch 4/100
13/13 [==============================] - 0s 405us/step - loss: 0.2567 - acc: 0.4615
Epoch 5/100
13/13 [========================

Epoch 79/100
13/13 [==============================] - 0s 351us/step - loss: 0.1252 - acc: 1.0000
Epoch 80/100
13/13 [==============================] - 0s 356us/step - loss: 0.1235 - acc: 1.0000
Epoch 81/100
13/13 [==============================] - 0s 329us/step - loss: 0.1217 - acc: 1.0000
Epoch 82/100
13/13 [==============================] - 0s 332us/step - loss: 0.1200 - acc: 1.0000
Epoch 83/100
13/13 [==============================] - 0s 319us/step - loss: 0.1182 - acc: 1.0000
Epoch 84/100
13/13 [==============================] - 0s 420us/step - loss: 0.1164 - acc: 1.0000
Epoch 85/100
13/13 [==============================] - 0s 352us/step - loss: 0.1146 - acc: 1.0000
Epoch 86/100
13/13 [==============================] - 0s 340us/step - loss: 0.1129 - acc: 1.0000
Epoch 87/100
13/13 [==============================] - 0s 368us/step - loss: 0.1112 - acc: 1.0000
Epoch 88/100
13/13 [==============================] - 0s 354us/step - loss: 0.1095 - acc: 1.0000
Epoch 89/100
13/13 [==========

Epoch 63/100
13/13 [==============================] - 0s 309us/step - loss: 0.1710 - acc: 0.9231
Epoch 64/100
13/13 [==============================] - 0s 310us/step - loss: 0.1696 - acc: 0.9231
Epoch 65/100
13/13 [==============================] - 0s 320us/step - loss: 0.1683 - acc: 0.9231
Epoch 66/100
13/13 [==============================] - 0s 252us/step - loss: 0.1669 - acc: 0.9231
Epoch 67/100
13/13 [==============================] - 0s 285us/step - loss: 0.1655 - acc: 0.9231
Epoch 68/100
13/13 [==============================] - 0s 289us/step - loss: 0.1641 - acc: 0.9231
Epoch 69/100
13/13 [==============================] - 0s 269us/step - loss: 0.1627 - acc: 0.9231
Epoch 70/100
13/13 [==============================] - 0s 295us/step - loss: 0.1613 - acc: 1.0000
Epoch 71/100
13/13 [==============================] - 0s 243us/step - loss: 0.1599 - acc: 1.0000
Epoch 72/100
13/13 [==============================] - 0s 304us/step - loss: 0.1586 - acc: 1.0000
Epoch 73/100
13/13 [==========

Epoch 47/100
13/13 [==============================] - 0s 326us/step - loss: 0.1530 - acc: 0.9231
Epoch 48/100
13/13 [==============================] - 0s 269us/step - loss: 0.1515 - acc: 0.9231
Epoch 49/100
13/13 [==============================] - 0s 323us/step - loss: 0.1499 - acc: 0.9231
Epoch 50/100
13/13 [==============================] - 0s 360us/step - loss: 0.1485 - acc: 0.9231
Epoch 51/100
13/13 [==============================] - 0s 434us/step - loss: 0.1471 - acc: 1.0000
Epoch 52/100
13/13 [==============================] - 0s 331us/step - loss: 0.1456 - acc: 1.0000
Epoch 53/100
13/13 [==============================] - 0s 266us/step - loss: 0.1442 - acc: 1.0000
Epoch 54/100
13/13 [==============================] - 0s 377us/step - loss: 0.1427 - acc: 1.0000
Epoch 55/100
13/13 [==============================] - 0s 370us/step - loss: 0.1412 - acc: 1.0000
Epoch 56/100
13/13 [==============================] - 0s 314us/step - loss: 0.1398 - acc: 1.0000
Epoch 57/100
13/13 [==========

In [14]:
    # csv書き出し
    PATH_RESULT = PATH + 'ACCURACY[loo]_timeseries_1dCNN.csv'
    result_loo.to_csv(PATH_RESULT, index = True)